<a href="https://colab.research.google.com/github/Danisa-R/Credit-Card-Fraud-Detector/blob/main/drunk_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!git clone https://github.com/ultralytics/yolov5 & cd yolov5 && pip install -r requirements.txt

/bin/bash: line 0: cd: yolov5: No such file or directory
Cloning into 'yolov5'...
remote: Enumerating objects: 15287, done.
remote: Total 15287 (delta 0), reused 0 (delta 0), pack-reused 15287
Receiving objects: 100% (15287/15287), 14.18 MiB | 12.94 MiB/s, done.
Resolving deltas: 100% (10489/10489), done.


In [3]:
import torch 
from matplotlib import pyplot as plt
import numpy as np
import cv2


In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s') # load pretained torchub model

/usr/local/lib/python3.8/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: YOLOv5 requirements "gitpython>=3.1.30" "setuptools>=65.5.1" not found, attempting AutoUpdate...


In [ ]:
model

## **Detect Image** 

In [ ]:
image = '/content/daddy_trudeau.jpeg'

In [ ]:
results = model(image)
results.print()

In [ ]:
%matplotlib inline
plt.imshow(np.squeeze(results.render()))
plt.show

In [ ]:
# need to access google colab specific API -- Colab code exectures on a VM that doesn't have webcam attached 
# (APIs that presume direct hardware access won't work)

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

## **Detect thru video** 

In [ ]:
import uuid
import os
import time

PATH_IMAGES = os.path.join('data', 'images') # return type: /data/images
labels = ['drunk', 'not drunk']
number_images = 20

# loop thru labels then loop thru images then take a picture to save inside data folder
video = cv2.VideoCapture(2) 
for label in labels:
  print("Collecting images for f"{labels})
  image_name=os.path.join(PATH_IMAGES, label+'.'+str(uuid.uuid1()+'.jpg'))

# saves an image to file b/c When working with OpenCV, images are stored in numpy ndarray
  for image_num in range(number_images):
    print("Collecting images for f"{label}" and f"{image_num}")

    # webcam feed
    check, frame = cap.read() 

    # writes out image to file
    cv2.imwrite(image_name, frame)

    # render to the screen
    cv2.imshow('Image Collection', frame) 

    time.sleep(2)

    # 2 sec delay between image captures
  if key & 0xFF == ord('q'): # hit 'q' on keyboard to quit
      break
video.release()
cv2.destroyAllWindows()



# TODO: might not use webcam to access data points, only use 
# access webcam
# while video.isOpened():
#   check, frame = video.read()
#   cv2.imshow('YOLO', frame)

#   # Make detecions
#   results=model(frame)

#   cv2.imshow('Color Frame', np.squeeze(results.render()))

#   # if check:
#   key = cv2.waitKey(10)
#   if key & 0xFF == ord('q'): # hit 'q' on keyboard to quit
#       break
#   # else:
#   #   print('Frame not available')
#   #   print(video.isOpened())

# video.release()
# cv2.destroyAllWindows()


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  